# Welcome to the movie rating and recommendation system!
# Step 1. import data 
1. u.data(user_rating_data): This is a tab separated list of user id | item id | rating | timestamp. 
2. u.user: this is a tab separated list of user id | age | gender | occupation | zip code
3. u.item: this is a tab separated
              list of
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |

## 1.1 Import and clean data

In [3]:
# related data 
import pandas as pd

# 1. import user rating data 
user_rating_data = pd.read_csv('u.data',delimiter="\t",header=None)
user_rating_data.columns=["userID","movieID","rating","timestamp"]
user_rating_data["timestamp"] = pd.to_datetime(user_rating_data["timestamp"],unit='s')
print(user_rating_data.head()) # contains user, movieID, rating, timestamp
# user_rating_data.to_csv("user_rating_data.csv",index=False)

# 2. import user demographic data 
user_demographic_data = pd.read_csv('u.user',delimiter="|",header=None) 
user_demographic_data.columns=["userID","age","gender","occupation","zipcode"]
print(user_demographic_data.head()) 
# user_demographic_data.to_csv("user_demographic_data.csv",index=False)

   userID  movieID  rating           timestamp
0     196      242       3 1997-12-04 15:55:49
1     186      302       3 1998-04-04 19:22:22
2      22      377       1 1997-11-07 07:18:36
3     244       51       2 1997-11-27 05:02:03
4     166      346       1 1998-02-02 05:33:16
   userID  age gender  occupation zipcode
0       1   24      M  technician   85711
1       2   53      F       other   94043
2       3   23      M      writer   32067
3       4   24      M  technician   43537
4       5   33      F       other   15213


In [4]:
# 3. import movie data
movie_data = pd.read_csv('u.item',delimiter="|",header=None,encoding='latin1')
movie_data.columns=["movieID","movieTitle","release date","video release date","IMDb URL","unknown","Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"]
movie_genre = ["Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"]

# add another column, each element is a list of generes 
movie_data["Genre"] = [[] for i in range(len(movie_data))] # a column is a list of values 
for genre in movie_genre:
    ids = [test1 for test1,test2 in zip(movie_data['movieID'],movie_data[genre]) if test2 == 1]
    for id in ids:
        # first select the genre column, then use lambda function, x is the current value of the genere column, assign the updated lists back to genre column 
        movie_data.loc[movie_data["movieID"]==id,'Genre'] = movie_data.loc[movie_data["movieID"]==id,'Genre'].apply(lambda x:x+[genre])

# filter the columns 
movie_data = movie_data[["movieID","movieTitle","Genre"]]
print(movie_data)
# movie_data.to_csv("movie_data.csv",index=False)

      movieID                                 movieTitle  \
0           1                           Toy Story (1995)   
1           2                           GoldenEye (1995)   
2           3                          Four Rooms (1995)   
3           4                          Get Shorty (1995)   
4           5                             Copycat (1995)   
...       ...                                        ...   
1677     1678                          Mat' i syn (1997)   
1678     1679                           B. Monkey (1998)   
1679     1680                       Sliding Doors (1998)   
1680     1681                        You So Crazy (1994)   
1681     1682  Scream of Stone (Schrei aus Stein) (1991)   

                                Genre  
0     [Animation, Children's, Comedy]  
1       [Action, Adventure, Thriller]  
2                          [Thriller]  
3             [Action, Comedy, Drama]  
4            [Crime, Drama, Thriller]  
...                               ...  

In [5]:
# review for zip
names = ["Alice","Tonny","John"]
ages = [20, 30, 40]
results = [name for name,age in zip(names,ages) if age>20]
print(results)

['Tonny', 'John']


In [6]:
# review for lamda 
data = pd.DataFrame({'numbers':[1,2,3,5]})
data['new_numbers'] = data['numbers'].apply(lambda x:x+100)
# x is the current value living in the data['numbers']
data
# lamda arguments: expression 

numbers  new_numbers
0        1          101
1        2          102
2        3          103
3        5          105

## 1.2 Import data into Neo4j

In [7]:
!pip install neo4j
import neo4j
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
def connect_db():
    driver = neo4j.GraphDatabase.driver(uri="neo4j://localhost:7687/?routing=direct", auth=("neo4j","password"))
    session = driver.session(database="neo4j")
    return session
    
def wipe_out_db(session):
    # wipe out database by deleting all nodes and relationships
    
    # similar to SELECT * FROM graph_db in SQL
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

session = connect_db()
wipe_out_db(session) 

In [9]:
def run_query_to_pandas(session, query, **kwargs): 
    # run a query and return the results in a pandas dataframe
    
    result = session.run(query, **kwargs) # dictionary of arguments
    # session.run(query, name='Tom Hanks', source='New York', target='Nashville', type='Person')
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

def run_query_print_raw(session, query, **kwargs):
    result = session.run(query,**kwargs)
    
    for r in result:
        print(r.values())
        

In [10]:
# idea: node a is user, node b is movie 
# 1. import user_rating_data 
query = '''
    LOAD CSV WITH HEADERS FROM 'file:///user_rating_data.csv' AS row
    MERGE (a:User {userID: row.userID})
    MERGE (b:Movie {movieID: row.movieID})
        
    
    MERGE (a)-[r:RATING]->(b)
        ON CREATE SET r.time = row.timestamp,
                      r.score = row.rating
        ON MATCH SET r.time = row.timestamp, 
                 r.score = row.rating 
    
'''
# node already exists, do on match; if the node does not exist, you will have on create- so on match will not execute when the relationships are newly created 
# code is a unique identifier 
session.run(query)

In [11]:
# 2. import user_demographic_data 
# each user node(a): add attribute: age, gender, occuptation, zipcode
query = '''
    LOAD CSV WITH HEADERS FROM 'file:///user_demographic_data.csv' AS row
    MERGE (a:User {userID: row.userID})
      ON CREATE SET a.age = row.age,
                    a.gender = row.gender,
                    a.occuptation = row.occupation,
                    a.zipcode = toInteger(row.zipcode)
      ON MATCH SET a.age = row.age,
                   a.gender = row.gender,
                   a.occupation = row.occupation,
                   a.zipcode = toInteger(row.zipcode)
'''

session.run(query)

# if merge find them, then create will be skipped 

In [12]:
# 3. import movie_data 
# each user node(b): add attribite: age, gender, occuptation, zipcode
query = '''
    LOAD CSV WITH HEADERS FROM 'file:///movie_data.csv' AS row
    MERGE (b:Movie {movieID: row.movieID})
      ON CREATE SET b.movieTitle = row.movieTitle,
                    b.Genre = row.Genre
      ON MATCH SET b.movieTitle = row.movieTitle,
                   b.Genre = row.Genre
'''

print(session.run(query))


In [13]:
query = '''
    MATCH (b:Movie)
    RETURN b.code
'''

# same as on create and on match 
# node already exists, do on match; if the node does not exist, you will have on create 
# better to input name 

run_query_print_raw(session,query)

[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]
[None]

## 1.3 Print all the nodes and relationships 

In [14]:
# print all the nodes and relationships"
   
print('-' * 20)
print("  Nodes:")
print('-' * 20)
    
query = '''
    match (n) 
    return n.name as node_name, labels(n) as labels
    order by n.name
'''
    
df_nodes = run_query_to_pandas(session, query)
num_nodes = df_nodes.shape[0]
display(df_nodes)
    
print('-' * 20)
print("  Relationships:")
print('-' * 20)
    
    
query = '''
    match (n1)-[r]->(n2) 
    return n1.name as n1_name, labels(n1) as n1_labels, 
        type(r) as relationship_type, n2.name as n2_name, labels(n2) as n2_labels
    order by n1_name, n2_name
'''
    
df_rels = run_query_to_pandas(session, query)
num_relationships = df_rels.shape[0]
display(df_rels)


--------------------
  Nodes:
--------------------


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: name)} {position: line: 3, column: 14, offset: 29} for query: '\n    match (n) \n    return n.name as node_name, labels(n) as labels\n    order by n.name\n'


node_name   labels
0         None   [User]
1         None  [Movie]
2         None   [User]
3         None  [Movie]
4         None   [User]
...        ...      ...
2620      None  [Movie]
2621      None  [Movie]
2622      None  [Movie]
2623      None  [Movie]
2624      None  [Movie]

[2625 rows x 2 columns]

--------------------
  Relationships:
--------------------


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: name)} {position: line: 3, column: 15, offset: 41} for query: '\n    match (n1)-[r]->(n2) \n    return n1.name as n1_name, labels(n1) as n1_labels, \n        type(r) as relationship_type, n2.name as n2_name, labels(n2) as n2_labels\n    order by n1_name, n2_name\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the datab

n1_name n1_labels relationship_type n2_name n2_labels
0        None    [User]            RATING    None   [Movie]
1        None    [User]            RATING    None   [Movie]
2        None    [User]            RATING    None   [Movie]
3        None    [User]            RATING    None   [Movie]
4        None    [User]            RATING    None   [Movie]
...       ...       ...               ...     ...       ...
99995    None    [User]            RATING    None   [Movie]
99996    None    [User]            RATING    None   [Movie]
99997    None    [User]            RATING    None   [Movie]
99998    None    [User]            RATING    None   [Movie]
99999    None    [User]            RATING    None   [Movie]

[100000 rows x 5 columns]

# Step 2 Develop new features for users 

In [15]:
from app import app
from bson.json_util import dumps, loads
from flask import request, jsonify
import json
import ast # helper library for parsing data from string
from importlib.machinery import SourceFileLoader

In [16]:
# feature: see the information about a movie, search by movie name
query = '''
    MATCH (b:Movie{movieTitle:"Toy Story (1995)"})
    RETURN b.movieTitle, b.movieID, b.Genre
'''
df = run_query_to_pandas(session, query)
display(df)

b.movieTitle b.movieID                                b.Genre
0  Toy Story (1995)         1  ['Animation', "Children's", 'Comedy']

In [17]:
# feature 0: see the information about a movie, search by movieID
query = '''
    MATCH (b:Movie{movieID:$movieID})
    RETURN b.movieID, b.movieTitle, b.Genre
'''
df = run_query_to_pandas(session, query,movieID = "3")
display(df)

# result = session.run(query, movieID = "1") 
# print(result.values())
# print(result.values())


b.movieID       b.movieTitle       b.Genre
0         3  Four Rooms (1995)  ['Thriller']

In [18]:
# feature 1. see the rating of specific user (search by userID) to a specific movie (search by movidTitle)
query = '''
    MATCH (a:User {userID: $userID}) -[r:RATING]->(b:Movie{movieID:$movieID})
    RETURN a.userID AS userID, r.score AS score, r.time AS time, b.movieID, b.movieTitle
'''
# df = run_query_to_pandas(session, query)
# display(df)
# result = session.run(query, userID = "196", movieID = "242") 
# for record in result.values():
#     print(record)

run_query_print_raw(session,query,userID = "196", movieID = "242")

['196', '3', '1997-12-04 15:55:49', '242', 'Kolya (1996)']


In [19]:
# feature 2. see the rating of a specific user (search by userID) to all the movies, ordered by rating
query='''
    MATCH (a:User {userID: $userID}) -[r:RATING]->(b:Movie)
    RETURN a.userID AS userID, r.score AS score, r.time AS time, b.movieID as movieID, b.movieTitle as title
    ORDER BY score DESC
'''
# df = run_query_to_pandas(session, query, userID = "196")
# display(df)

result = session.run(query, userID = "196") 
for record in result.values():
    print(record)

['196', '5', '1997-12-04 16:13:37', '692', 'American President, The (1995)']
['196', '5', '1997-12-04 16:11:51', '663', 'Being There (1979)']
['196', '5', '1997-12-04 16:09:13', '8', 'Babe (1995)']
['196', '5', '1997-12-04 16:10:20', '153', 'Fish Called Wanda, A (1988)']
['196', '5', '1997-12-04 16:09:53', '655', 'Stand by Me (1986)']
['196', '5', '1997-12-04 16:09:13', '285', 'Secrets & Lies (1996)']
['196', '5', '1997-12-04 15:55:49', '286', 'English Patient, The (1996)']
['196', '5', '1997-12-04 16:13:37', '67', 'Ace Ventura: Pet Detective (1994)']
['196', '4', '1997-12-04 16:15:28', '1118', 'Up in Smoke (1978)']
['196', '4', '1997-12-04 16:10:20', '238', 'Raising Arizona (1987)']
['196', '4', '1997-12-04 16:11:03', '393', 'Mrs. Doubtfire (1993)']
['196', '4', '1997-12-04 15:57:01', '306', 'Mrs. Brown (Her Majesty, Mrs. Brown) (1997)']
['196', '4', '1997-12-04 16:15:10', '108', 'Kids in the Hall: Brain Candy (1996)']
['196', '4', '1997-12-04 16:08:48', '381', "Muriel's Wedding (1994

## Enable the user to update a rating for a movie

In [28]:
# feature: enable the user to update a new rating for a movie, userID is 196, the movieID is 1, the score to add is 3 
# if there is an old rating, update it; if there is no rating, add that
query = '''
    MERGE (a:User {userID:$userID})-[r:RATING]->(b:Movie {movieID:$movieID})
        ON MATCH SET r.time = datetime(), r.score = $rating
        ON CREATE SET r.time = datetime(), r.score = $rating
    
'''
session.run(query,userID="196",movieID="242",rating=5)

In [29]:
# check whether it is added or updated 
query = '''
    MATCH (a:User {userID: '196'}) -[r:RATING]->(b:Movie{movieID:"242"})
    RETURN a.userID AS userID, r.score AS score, r.time AS time, b.movieID, b.movieTitle
'''
run_query_print_raw(session,query)

['196', 5, neo4j.time.DateTime(2024, 12, 2, 1, 53, 9, 958000000, tzinfo=<UTC>), '242', 'Kolya (1996)']


## Enable the user to delete a rating for a movie

In [30]:
# feature: enable the user to delete the rating for a movie, userid is 196, movie is 1 
query = '''
    MATCH (a:User {userID: $userID})-[r:RATING]->(b:Movie {movieID: $movieID})
    DELETE r
'''
# remember to first use match, then use delete 
session.run(query,userID="196",movieID="242")

In [31]:
# check whether it is deleted -YES
query = '''
    MATCH (a:User {userID: $userID}) -[r:RATING]->(b:Movie{movieID:$movieID})
    RETURN a.userID AS userID, r.score AS score, r.time AS time, b.movieID, b.movieTitle
'''
df = run_query_to_pandas(session, query,userID='196',movieID="242")
display(df)

Empty DataFrame
Columns: [userID, score, time, b.movieID, b.movieTitle]
Index: []

## Enable the user to check the average ratings for all the movies and know which movie has the highest rating

In [73]:
# feature: 
query = '''
    MATCH (a:User) -[r:RATING]->(b:Movie)
    WITH b.movieID AS movieID, b.movieTitle as movieTitle, AVG(toFloat(r.score)) AS averageScore
    ORDER BY averageScore DESC
    RETURN movieID, movieTitle, averageScore
    
'''
df = run_query_to_pandas(session, query)
display(df)
# grouping data is implied by the use of the WITH clause along with an aggregation function, such as AVG(), COUNT(), SUM(), etc. The WITH clause determines the grouping by the fields specified before the aggregation. 

movieID                            movieTitle  averageScore
0        242                                  None           5.0
1       1467  Saint of Fort Washington, The (1993)           5.0
2       1189                    Prefontaine (1997)           5.0
3        814         Great Day in Harlem, A (1994)           5.0
4       1122        They Made Me a Criminal (1939)           5.0
...      ...                                   ...           ...
1678    1654          Chairman of the Board (1998)           1.0
1679    1659       Getting Away With Murder (1996)           1.0
1680    1661                   New Age, The (1994)           1.0
1681    1671             Further Gesture, A (1996)           1.0
1682    1678                     Mat' i syn (1997)           1.0

[1683 rows x 3 columns]

In [ ]:
# feature: check the rating for one movie 
query = '''
    MATCH (a:User) -[r:RATING]->(b:Movie{movieID:$movieID})
    RETURN b.movieID AS movieID, b.movieTitle as movieTitle, AVG(toFloat(r.score)) AS averageScore
'''
df = run_query_to_pandas(session, query,movieID="1678")
display(df)

movieID         movieTitle  averageScore
0    1678  Mat' i syn (1997)           1.0

## Enable the user to check the average ratings for movie of different Genres

In [ ]:
# feature: 
query = '''
    MATCH (a:User) -[r:RATING]->(b:Movie)
    WITH b.Genre AS Genre, AVG(toFloat(r.score)) AS averageScore
    ORDER BY averageScore DESC
    RETURN Genre, averageScore
    
'''
df = run_query_to_pandas(session, query)
display(df)
# grouping data is implied by the use of the WITH clause along with an aggregation function, such as AVG(), COUNT(), SUM(), etc. The WITH clause determines the grouping by the fields specified before the aggregation. 

Genre  averageScore
0                  ['Animation', 'Comedy', 'Thriller']      4.491071
1                                        ['Animation']      4.430556
2                                    ['Sci-Fi', 'War']      4.252577
3                               ['Crime', 'Film-Noir']      4.250000
4    ['Action', 'Adventure', 'Drama', 'Romance', 'S...      4.204360
..                                                 ...           ...
211                           ['Action', "Children's"]      2.080000
212    ['Action', 'Adventure', "Children's", 'Sci-Fi']      2.076923
213                          ["Children's", 'Fantasy']      2.066667
214              ['Action', 'Adventure', "Children's"]      1.800000
215                                        ['Fantasy']      1.000000

[216 rows x 2 columns]

# Step.3

## 2.1 Calculate the similarity scores between users using Jaccard similarity
Jaccard similarity is the ratio of the number of nodes that the two users are both connected with divided by the number of all the nodes that one or two users relate to. This assumes that users watch what they like and then rate. For instance, user A and user B both rated movie A. User C rated movie A and movie B. Therefore, the similarity between user A and user B is 100%. The similarity between User B and user C is 50%.

In [ ]:
# feature: 
# user 196 and user 132
query = '''
    MATCH (a:User{userID:'196'}) -[r:RATING]->(b:Movie)
    RETURN a.userID, b.movieID
    
'''
df = run_query_to_pandas(session, query)
display(df)

a.userID b.movieID
0       196       238
1       196      1022
2       196        13
3       196         8
4       196        67
5       196       411
6       196       153
7       196       251
8       196        70
9       196       202
10      196        25
11      196       110
12      196       111
13      196       762
14      196       285
15      196       692
16      196       845
17      196       428
18      196      1007
19      196       580
20      196      1118
21      196       340
22      196       381
23      196        66
24      196       287
25      196       382
26      196       393
27      196       257
28      196       116
29      196       655
30      196       663
31      196       306
32      196       173
33      196        94
34      196      1241
35      196       108
36      196       269
37      196       242
38      196       286

In [ ]:
# feature: 
# user 196 and user 132
query = '''
    MATCH (a:User{userID:'132'}) -[r:RATING]->(b:Movie)
    RETURN a.userID,b.movieID
    
'''
df = run_query_to_pandas(session, query)
display(df)

a.userID b.movieID
0       132        56
1       132       175
2       132        50
3       132       484
4       132       154
5       132       275
6       132       664
7       132       806
8       132       521
9       132       523
10      132       124
11      132       100
12      132       922
13      132      1019
14      132       251
15      132        12
16      132       285
17      132       137
18      132      1154
19      132       286
20      132       127
21      132       151

In [ ]:
# check the overlapping nodes- this is the number 
# feature: 
# user 196 and user 132
query = '''
    MATCH (a1:User{userID:'132'}) -[:RATING]->(b:Movie)<-[:RATING]-(a2:User{userID:'196'})
    WITH COUNT(DISTINCT b) AS overlappingMovies
    MATCH (a:User)-[:RATING]->(c:Movie)
    WHERE a.userID IN ['196', '132']
    WITH overlappingMovies, COUNT(DISTINCT c) AS totalMovies
    RETURN toFloat(overlappingMovies)/totalMovies AS JScore
'''
run_query_print_raw(session, query)
    
# grouping data is implied by the use of the WITH clause along with an aggregation function, such as AVG(), COUNT(), SUM(), etc. The WITH clause determines the grouping by the fields specified before the aggregation. 
# while WITH allows you to pass intermediate results forward to subsequent clauses.

[0.05172413793103448]


## 2.2 Calculate the weighted sum

In [ ]:
# think: give user 196 recommendation of movies, for each movie, there will be a score, the score is the weighted sum of others' ratings
# for movie 70

# for userid 135
# weight
query = '''
    MATCH (a1:User{userID:'135'}) -[:RATING]->(b:Movie)<-[:RATING]-(a2:User{userID:'196'})
    WITH COUNT(DISTINCT b) AS overlappingMovies
    MATCH (a:User)-[:RATING]->(c:Movie)
    WHERE a.userID IN ['135', '196']
    WITH overlappingMovies, COUNT(DISTINCT c) AS totalMovies
    RETURN toFloat(overlappingMovies)/totalMovies AS JScore
'''
result1 = session.run(query).values()

# score
query = '''
    MATCH (a:User{userID:'135'}) -[r:RATING]->(b:Movie{movieID:"77"})
    RETURN r.score
'''
result1_1 = session.run(query).values()

# for user 126
query = '''
    MATCH (a1:User{userID:'136'}) -[:RATING]->(b:Movie)<-[:RATING]-(a2:User{userID:'196'})
    WITH COUNT(DISTINCT b) AS overlappingMovies
    MATCH (a:User)-[:RATING]->(c:Movie)
    WHERE a.userID IN ['136', '196']
    WITH overlappingMovies, COUNT(DISTINCT c) AS totalMovies
    RETURN toFloat(overlappingMovies)/totalMovies AS JScore
'''
result2 = session.run(query).values()

query = '''
    MATCH (a:User{userID:'136'}) -[r:RATING]->(b:Movie{movieID:"70"})
    RETURN r.score
'''
result2_1 = session.run(query).values()

# check
print(result1)
print(result1_1)
print(result2)
print(result2_1)

# print(type(result1 [0][0]))
check = result1[0][0] * float(result1_1[0][0])
check


[[0.010752688172043012]]
[['4']]
[[0.05714285714285714]]
[]


0.043010752688172046

In [ ]:
# for user 126
query = '''
    MATCH (a1:User{userID:'136'}) -[:RATING]->(b:Movie)<-[:RATING]-(a2:User{userID:'196'})
    WITH COUNT(DISTINCT b) AS overlappingMovies
    MATCH (a:User)-[:RATING]->(c:Movie)
    WHERE a.userID IN ['136', '196']
    WITH overlappingMovies, COUNT(DISTINCT c) AS totalMovies
    RETURN toFloat(overlappingMovies)/totalMovies AS JScore
'''
result = session.run(query)
print(result.values())


[[0.05714285714285714]]


In [ ]:
# score
query = '''
    MATCH (a:User) -[r:RATING]->(b:Movie{movieID:"285"})
    RETURN a.userID, r.score, b.movieID
'''
run_query_to_pandas(session,query)


a.userID r.score b.movieID
0        463       4       285
1        370       3       285
2        765       5       285
3        492       4       285
4        406       5       285
..       ...     ...       ...
157      468       4       285
158      379       5       285
159      699       4       285
160      591       5       285
161      524       3       285

[162 rows x 3 columns]

In [ ]:
# score
query = '''
    MATCH (a:User) -[r:RATING]->(b:Movie{movieID:"77"})
    RETURN a.userID, r.score, b.movieID
'''
run_query_to_pandas(session,query)


a.userID r.score b.movieID
0        478       1        77
1        135       4        77
2        244       4        77
3        487       3        77
4        394       3        77
..       ...     ...       ...
146      416       4        77
147      659       4        77
148      313       3        77
149      110       4        77
150      286       3        77

[151 rows x 3 columns]

In [ ]:
# recap1- movieID-285- this one works!
users_all = list(range(1,944)) # 1 to 943, this is a list, each one is integer
users_all = list(map(str, users_all)) # each one is a string now 
user_input = "196"
other_user = [users for users in users_all if users !=user_input]
print("other_user",other_user)


# for userid 196
# weight
recommendationScore = 0 
for userID_each in other_user:
    query = '''
        MATCH (a1:User{userID:'196'}) -[:RATING]->(b:Movie{movieID:"285"})<-[:RATING]-(a2:User{userID: $userID})
        WITH COUNT(DISTINCT b) AS overlappingMovies
        MATCH (a:User)-[:RATING]->(c:Movie)
        WHERE a.userID IN [$userID, '196']
        WITH overlappingMovies, COUNT(DISTINCT c) AS totalMovies
        RETURN toFloat(overlappingMovies)/totalMovies AS JScore
    '''
    weight = session.run(query,userID = userID_each).values()
    print("weight",weight[0][0])
    weight_final = weight[0][0]

    # score
    query = '''
        MATCH (a:User{userID:$userID}) -[r:RATING]->(b:Movie{movieID:"285"})
        RETURN r.score
    '''
    score = session.run(query,userID=userID_each).values()
    print("here",score)
    print("Current userID_each:", userID_each)
    if score: # if not none
        print("here",score[0][0])
        score_final = float(score[0][0])
    else:
        score_final = 0 
    
    # add to recommendation score(=weighted sum)
    recommendationScore += weight_final * score_final


# print the weighted sum
print(recommendationScore)


other_user ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157

In [ ]:
# recap2- movieID-285- this one works!
users_all = list(range(1,944)) # 1 to 943, this is a list, each one is integer
users_all = list(map(str, users_all)) # each one is a string now 
user_input = "196"
other_user = [users for users in users_all if users !=user_input]
print("other_user",other_user)


# for userid 196
# weight
recommendationScore = 0 
for userID_each in other_user:
    query = '''
        MATCH (a1:User{userID:'196'}) -[:RATING]->(b:Movie{movieID:$movieID})<-[:RATING]-(a2:User{userID: $userID})
        WITH COUNT(DISTINCT b) AS overlappingMovies
        MATCH (a:User)-[:RATING]->(c:Movie)
        WHERE a.userID IN [$userID, '196']
        WITH overlappingMovies, COUNT(DISTINCT c) AS totalMovies
        RETURN toFloat(overlappingMovies)/totalMovies AS JScore
    '''
    weight = session.run(query,userID = userID_each,movieID="285").values()
    # print("weight",weight[0][0])
    weight_final = weight[0][0]

    # score
    query = '''
        MATCH (a:User{userID:$userID}) -[r:RATING]->(b:Movie{movieID:$movieID})
        RETURN r.score
    '''
    score = session.run(query,userID=userID_each, movieID="285").values()
    # print("here",score)
    # print("Current userID_each:", userID_each)
    if score: # if not none
        # print("here",score[0][0])
        score_final = float(score[0][0])
    else:
        score_final = 0 
    
    # add to recommendation score(=weighted sum)
    recommendationScore += weight_final * score_final


# print the weighted sum
print(recommendationScore)


other_user ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157

In [ ]:
# recap3! recommentation a list of movies 
users_all = list(range(1,944)) # 1 to 943, this is a list, each one is integer
users_all = list(map(str, users_all)) # each one is a string now 
user_input = "196"
other_user_list = [users for users in users_all if users !=user_input]
# movies_selected = list(map(str, [284,285,286]))
# Prompt user to input up to 5 movies
movie_ids = input("Enter up to 5 movie IDs (comma-separated): ").split(",")[:5] #.split(","):Splits the string into a list of strings wherever it finds a comma.
movie_ids = [movie.strip() for movie in movie_ids] #movie.strip removes any space, ' 123' will be '123'

print("other_user",other_user)

for movie_each in movie_ids:
    recommendationScore = 0 
    for userID_each in other_user:
        query = '''
            MATCH (a1:User{userID:'196'}) -[:RATING]->(b:Movie{movieID:$movieID})<-[:RATING]-(a2:User{userID: $userID})
            WITH COUNT(DISTINCT b) AS overlappingMovies
            MATCH (a:User)-[:RATING]->(c:Movie)
            WHERE a.userID IN [$userID, '196']
            WITH overlappingMovies, COUNT(DISTINCT c) AS totalMovies
            RETURN toFloat(overlappingMovies)/totalMovies AS JScore
        '''
        weight = session.run(query,userID = userID_each,movieID=movie_each).values()
        # print("weight",weight[0][0])
        weight_final = weight[0][0]

        # score
        query = '''
            MATCH (a:User{userID:$userID}) -[r:RATING]->(b:Movie{movieID:$movieID})
            RETURN r.score
        '''
        score = session.run(query,userID=userID_each, movieID=movie_each).values()
        # print("here",score)
        # print("Current userID_each:", userID_each)
        if score: # if not none
            # print("here",score[0][0])
            score_final = float(score[0][0])
        else:
            score_final = 0 
        
        # add to recommendation score(=weighted sum)
        recommendationScore += weight_final * score_final


    # print the weighted sum
    print(movie_each, recommendationScore)

   

other_user ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157

In [47]:
from neo4j import GraphDatabase

# Initialize Neo4j driver
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))  # Update with your credentials

def getWeight(userInput, otherUser, movieID):
    with driver.session() as session:
        query = '''
            MATCH (a1:User {userID: $userInputID})-[:RATING]->(b:Movie {movieID: $movieID})<-[:RATING]-(a2:User {userID: $otherUserID})
            WITH COUNT(DISTINCT b) AS overlappingMovies
            MATCH (a:User)-[:RATING]->(c:Movie)
            WHERE a.userID IN [$otherUserID, $userInputID]
            WITH overlappingMovies, COUNT(DISTINCT c) AS totalMovies
            RETURN toFloat(overlappingMovies)/totalMovies AS JScore
        '''
        result = session.run(query, userInputID=userInput, otherUserID=otherUser, movieID=movieID)
        record = result.single()
        return record["JScore"] if record else 0.0

def getRating(otherUser, movieID):
    with driver.session() as session:
        query = '''
            MATCH (a:User {userID: $userID})-[r:RATING]->(b:Movie {movieID: $movieID})
            RETURN r.score AS score
        '''
        result = session.run(query, userID=otherUser, movieID=movieID)
        record = result.single()
        return float(record["score"]) if record else 0.0

# Iterate through each movie to calculate recommendation scores
movies_selected = ["285", "7", "9"]
user_input = "196"
other_user_list = [str(i) for i in range(1, 944) if str(i) != user_input]
print(other_user_list)

all_movie_recommendation_scores = []

for movie_each in movies_selected:
    print(f"Processing movie {movie_each}...")
    recommendationScore = 0

    for userID_each in other_user_list:
        try:
            weight = getWeight(user_input, userID_each, movie_each)
            score = getRating(userID_each, movie_each)
            recommendationScore += weight * score
        except Exception as e:
            print(f"Error processing user {userID_each} for movie {movie_each}: {e}")
    
    print(f"Recommendation score for movie {movie_each}: {recommendationScore}")
    all_movie_recommendation_scores.append((movie_each, recommendationScore))

# Close the driver connection
driver.close()

# Sort and display top recommendations
all_movie_recommendation_scores.sort(key=lambda x: x[1], reverse=True)

print("\nTop Recommendations:")
for movie, score in all_movie_recommendation_scores:
    print(f"Movie ID: {movie}, Score: {score}")


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '

# Step 4. additional features

## Enable the user to know the most frequently rated movies

In [49]:
query = '''
    MATCH (b:Movie)-[r:RATING]-(a:User)
    WITH b.movieID AS movieID, b.movieTitle AS movieTitle, COUNT(r) AS ratingCount
    RETURN movieID, movieTitle, ratingCount
    ORDER BY ratingCount DESC
'''
run_query_to_pandas(session,query)

movieID                                 movieTitle  ratingCount
0         50                           Star Wars (1977)          583
1        258                             Contact (1997)          509
2        100                               Fargo (1996)          508
3        181                  Return of the Jedi (1983)          507
4        294                           Liar Liar (1997)          485
...      ...                                        ...          ...
1678    1680                       Sliding Doors (1998)            1
1679    1678                          Mat' i syn (1997)            1
1680    1679                           B. Monkey (1998)            1
1681    1681                        You So Crazy (1994)            1
1682    1682  Scream of Stone (Schrei aus Stein) (1991)            1

[1683 rows x 3 columns]

## Recommend the user to follow others who may have the same taste 
User can choose how many movies they rate in common and the number of other users they are recommended. The smaller the total difference, the more similar they are. 

In [68]:
query = '''
    MATCH (a1:User {userID: $userID})-[r1:RATING]->(m:Movie)<-[r2:RATING]-(a2:User)
    WHERE a1 <> a2
    WITH a2.userID AS similarUserID, SUM(ABS(toFloat(r1.score) - toFloat(r2.score))) AS totalDifference, COUNT(m) as numberCommonMovies
    WHERE numberCommonMovies >= $movieInCommon 
    ORDER BY totalDifference ASC
    RETURN similarUserID, totalDifference
    LIMIT $numberUsers
'''
run_query_to_pandas(session,query,userID="196",movieInCommon=10,numberUsers=10)

similarUserID  totalDifference
0           665              5.0
1           634              6.0
2           624              7.0
3           243              7.0
4           532              8.0
5           764              8.0
6           119              8.0
7           629              8.0
8           436              8.0
9           360              9.0